In [1]:
# Imports
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import Vectorizer
from Common_Time import tunes as tunes_raw
BAR_SUBDIVISION = 48
NOTE_MULT = BAR_SUBDIVISION//8 #value to multiply an 8th note by 
PAD_METHOD = 1 # 1 is stretch

In [2]:
tunes = pd.DataFrame.from_dict(tunes_raw, orient='index')
tunes['abc_raw'] = tunes.abc # preserve the original abc strings 
tunes.head(10)

,setting,type,mode,meter,tune,abc,abc_raw
1,1,reel,Edorian,4/4,1,D2|EBBAB2EB|B2ABdBAG|FDADBDAD|FDADdAFD|EBBAB2E...,D2|EBBAB2EB|B2ABdBAG|FDADBDAD|FDADdAFD|EBBAB2E...
2,2,reel,Dmajor,4/4,2,A2FAA2dB|A2FABEE2|A2FAA2Bd|egfdedBd|A2FAA2dB|A...,A2FAA2dB|A2FABEE2|A2FAA2Bd|egfdedBd|A2FAA2dB|A...
3,3,reel,Gmajor,4/4,3,BA|G2BGAFDF|G2Bdc2BA|G2BGAFDF|GEEGc2BA|G2BGAFD...,BA|G2BGAFDF|G2Bdc2BA|G2BGAFDF|GEEGc2BA|G2BGAFD...
4,4,hornpipe,Dmajor,4/4,4,ag|faefdeAF|DFAdf2ef|gbegdeAF|GBAGE2ag|faefdeA...,ag|faefdeAF|DFAdf2ef|gbegdeAF|GBAGE2ag|faefdeA...
7,7,hornpipe,Adorian,4/4,7,EG|A2ABAGE2|cdecd2eg|agedcded|cAGEG2cB|A2ABAGE...,EG|A2ABAGE2|cdecd2eg|agedcded|cAGEG2cB|A2ABAGE...
8,8,reel,Gmajor,4/4,8,G2GDEDEG|AGABd2Bd|egedBAGA|BAGEEDDE|G2GDEDEG|A...,G2GDEDEG|AGABd2Bd|egedBAGA|BAGEEDDE|G2GDEDEG|A...
13,13,hornpipe,Dmajor,4/4,13,de|fefgafge|fdecdBAG|FEDEFGAB|=cAd^cA2de|fefga...,de|fefgafge|fdecdBAG|FEDEFGAB|=cAd^cA2de|fefga...
16,16,reel,Amajor,4/4,16,ceefecBA|dfff=gfed|ceefecBA|dBBAB2d2|ceefecBA|...,ceefecBA|dfff=gfed|ceefecBA|dBBAB2d2|ceefecBA|...
18,18,reel,Dmajor,4/4,18,A2FABAFA|A2FABAFA|B2BAB2BA|B2BABAFA|A2FABAFA|A...,A2FABAFA|A2FABAFA|B2BAB2BA|B2BABAFA|A2FABAFA|A...
21,21,reel,Aminor,4/4,21,A2cAABcA|GcEGG2EG|A2cAABcd|ecdBcAA2|A2cAABcA|G...,A2cAABcA|GcEGG2EG|A2cAABcd|ecdBcAA2|A2cAABcA|G...


In [3]:
def split_by_bar(abc_string):
    """
    Takes a string of full ABC notation and splits it into lists representing individual bars
    """
    result = abc_string.split('|')
    return [x for x in result if x != '']

In [4]:
note_numbers = {
    'C' : 60,
    'D' : 62,
    'E' : 64,
    'F' : 65,
    'G' : 67,
    'A' : 69,
    'B' : 71,
    'c' : 72,
    'd' : 74,
    'e' : 76,
    'f' : 77,
    'g' : 79,
    'a' : 81,
    'b' : 83,
    'z' : 0
}

accidentals = {'_' : -1, '^' : 1}
octaves = {',':-12, "'":12}

def note_to_number(abc_note):
    reg = re.search('([_=^]*)([a-gzA-G])([,\']*)', abc_note)
    accidental = reg.group(1)
    note = reg.group(2)
    octave = reg.group(3)
    val = note_numbers[note]
    if accidental in accidentals:
        val += accidentals[accidental]
    for char in octave:
        val+= octaves[char]
    return val     

In [5]:
def vectorize_bar(abc_string, pad_bars=True):
    """
    Takes an ABC string of an individual bar and returns that same bar but with 48 notes
    returns the the string of notes as well as a 'timing vector'
    """
    note_out = []
    time_out = []
    notes_re = re.compile('([_=^]*[a-gzA-G][,\']*)([\d]*)/?([\d]*)')
    notes = notes_re.findall(abc_string)
    for note, num, denom in notes:
        if num == '': num=1
        else: num = int(num)
        if denom == '': denom=1
        else: denom = int(denom)
        if NOTE_MULT*num/denom%1 != 0:
            return np.array(note_out), np.array(time_out)
            raise Exception("{}*{}/{} is not an integer. Your note multiplier is insufficient".format(NOTE_MULT,num,denom))
        reps = ((NOTE_MULT*num)//denom)
        note_num = note_to_number(note)
        note_out += [note_num for n in range(reps)]
        time = [0]*reps
        time[0]=1
        time_out+=time  
    # zero padding
    if pad_bars and len(note_out) != BAR_SUBDIVISION:
        pad_num = BAR_SUBDIVISION - len(note_out)
        if len(notes)<4:
            note_out += [0 for n in range(pad_num)]
            time_out += [0 for n in range(pad_num)]
        else:
            note_out = [0 for n in range(pad_num)] + note_out
            note_out = [0 for n in range(pad_num)] + note_out
    return np.array(note_out), np.array(time_out)

In [6]:
def vectorize_abc(abc_string):
    """
    takes an abc string and returns the note vector and timing vector, split by bars
    """
    note_out = []
    time_out = []
    for bar in split_by_bar(abc_string):
        note, time = vectorize_bar(bar)
        note_out.append(note)
        time_out.append(time)
    return np.array(note_out), np.array(time_out)

<h1>Visualizing

In [ ]:
def print_img_by_bars(songs, times, numcols=3):
    """
    Takes an array of arrays of size 16,16 and plots it in 3 configurations
    """
    f, axs = plt.subplots(len(songs),numcols,figsize=(10,len(songs)*4))
    plt.tight_layout()
    for num, song in enumerate(songs):
        img = song[:16] 
#         img+= times[num]*10
        
        rgbArray = np.zeros((16,48,3))
        rgbArray[..., 0] =  times[num]
        rgbArray[..., 1] = img/10
        rgbArray[..., 2] = img/10
        img = rgbArray
        
        index = num*numcols+1
        plt.subplot(len(songs), 3, index)
        plt.axis('off')
        plt.title("1 bar per line", fontsize=10)
        plt.imshow(img, cmap='gray',aspect="auto", vmin=60)
        plt.subplot(len(songs), 3, index+1)
        plt.axis('off')
        plt.title("2 bars per line", fontsize=10)
        plt.imshow(img.reshape(8,96,3), cmap='gray', interpolation='nearest', aspect="auto")
        plt.subplot(len(songs), 3, index+2)
        plt.axis('off')
        plt.title("4 bars per line", fontsize=10)
        plt.imshow(img.reshape(4,192,3), cmap='gray', interpolation='nearest', aspect="auto", vmin=60)

In [ ]:
sample = tunes.sample(10)

In [ ]:
song_graph = []
time_graph = []
for song in sample['abc']:
    notes, time = vectorize_abc(song)
    if len(notes.shape) is 1:
        continue
    else:
        song_graph.append(notes[:16])
        time_graph.append(time[:16])
print_img_by_bars(song_graph, time_graph, numcols=3)

In [7]:
def vectorize_frame(df):
    """
    Takes a pandas dataframe and returns it with 2 new frames appended for notes and timing
    :param df: pandas dataframe with an 'abc' column
    """
    df['notes'], df['timing'] = zip(*df.abc.map(vectorize_abc))

In [8]:
vectorize_frame(tunes)

In [10]:
tunes.reset_index(drop=True, inplace=True)
tunes

,setting,type,mode,meter,tune,abc,abc_raw,notes,timing
0,1,reel,Edorian,4/4,1,D2|EBBAB2EB|B2ABdBAG|FDADBDAD|FDADdAFD|EBBAB2E...,D2|EBBAB2EB|B2ABdBAG|FDADBDAD|FDADdAFD|EBBAB2E...,"[[62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,2,reel,Dmajor,4/4,2,A2FAA2dB|A2FABEE2|A2FAA2Bd|egfdedBd|A2FAA2dB|A...,A2FAA2dB|A2FABEE2|A2FAA2Bd|egfdedBd|A2FAA2dB|A...,"[[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,3,reel,Gmajor,4/4,3,BA|G2BGAFDF|G2Bdc2BA|G2BGAFDF|GEEGc2BA|G2BGAFD...,BA|G2BGAFDF|G2Bdc2BA|G2BGAFDF|GEEGc2BA|G2BGAFD...,"[[71, 71, 71, 71, 71, 71, 69, 69, 69, 69, 69, ...","[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,4,hornpipe,Dmajor,4/4,4,ag|faefdeAF|DFAdf2ef|gbegdeAF|GBAGE2ag|faefdeA...,ag|faefdeAF|DFAdf2ef|gbegdeAF|GBAGE2ag|faefdeA...,"[[81, 81, 81, 81, 81, 81, 79, 79, 79, 79, 79, ...","[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,7,hornpipe,Adorian,4/4,7,EG|A2ABAGE2|cdecd2eg|agedcded|cAGEG2cB|A2ABAGE...,EG|A2ABAGE2|cdecd2eg|agedcded|cAGEG2cB|A2ABAGE...,"[[64, 64, 64, 64, 64, 64, 67, 67, 67, 67, 67, ...","[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
5,8,reel,Gmajor,4/4,8,G2GDEDEG|AGABd2Bd|egedBAGA|BAGEEDDE|G2GDEDEG|A...,G2GDEDEG|AGABd2Bd|egedBAGA|BAGEEDDE|G2GDEDEG|A...,"[[67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
6,13,hornpipe,Dmajor,4/4,13,de|fefgafge|fdecdBAG|FEDEFGAB|=cAd^cA2de|fefga...,de|fefgafge|fdecdBAG|FEDEFGAB|=cAd^cA2de|fefga...,"[[74, 74, 74, 74, 74, 74, 76, 76, 76, 76, 76, ...","[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
7,16,reel,Amajor,4/4,16,ceefecBA|dfff=gfed|ceefecBA|dBBAB2d2|ceefecBA|...,ceefecBA|dfff=gfed|ceefecBA|dBBAB2d2|ceefecBA|...,"[[72, 72, 72, 72, 72, 72, 76, 76, 76, 76, 76, ...","[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,..."
8,18,reel,Dmajor,4/4,18,A2FABAFA|A2FABAFA|B2BAB2BA|B2BABAFA|A2FABAFA|A...,A2FABAFA|A2FABAFA|B2BAB2BA|B2BABAFA|A2FABAFA|A...,"[[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
9,21,reel,Aminor,4/4,21,A2cAABcA|GcEGG2EG|A2cAABcd|ecdBcAA2|A2cAABcA|G...,A2cAABcA|GcEGG2EG|A2cAABcd|ecdBcAA2|A2cAABcA|G...,"[[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, ...","[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
